In [1]:
from io import BytesIO

from IPython.core import display
from PIL import Image


def display_pil_image(im):
   """Displayhook function for PIL Images, rendered as PNG."""

   b = BytesIO()
   im.save(b, format='png')
   data = b.getvalue()

   ip_img = display.Image(data=data, format='png', embed=True)
   return ip_img._repr_png_()


# register display func with PNG formatter:
png_formatter = get_ipython().display_formatter.formatters['image/png']
dpi = png_formatter.for_type(Image.Image, display_pil_image)

In [2]:
from PIL import Image
import re
img = {'size': (1500, 2247), 'source': '/srv/fotoarkiv_universitas/byline/Louise-Faldalen-Prytz.jpg', 'options': {}} 
geometry = (200,200)
options = {'padding': False, 'format': 'JPEG', 'image_info': {}, 
           'colorspace': 'RGB', 'quality': 95, 'upscale': True, 
           'rounded': None, 'crop': '31% 41%', 'cropbox': None, 'diameter': 31, 
           'padding_color': '#ffffff'}


def close_crop(self, image, geometry, options):
    """ crop it close """
    # crop circle diameter in pixels
    org_width, org_height = size = image['size']
    filename = image['source']
    diameter = options['diameter'] * min(size) / 100 * 1.5
    crop = options['crop']
    if type(crop) is str and '%' in crop:
        
        ratio = org_width/org_height
        new_ratio = geometry[0]/geometry[1]
        if new_ratio > 1:
            #landscape
            width, height = diameter * new_ratio, diameter
        else:
            # portrait
            width, height = diameter, diameter / new_ratio
        
        if org_width > width and org_height > height:
            top, left = [int(match) for match in re.findall(r'\d+', crop)]
            left = left * org_width / 100
            top = top * org_height / 100
            print(top, left)
            crop_top = min(org_height - height, max(0, top - height/2))
            crop_left = min(org_width - width, max(0, left - width/2))
            crop_right = crop_left + width
            crop_bottom = crop_top + height
            
            new_geometry = [int(value) for value in (crop_left, crop_top, crop_right, crop_bottom)]
            
            original = Image.open(filename)
            cropped = original.crop(new_geometry)
            cropped.thumbnail(geometry)
            cropped.save(filename, **options)
            image['size'] = geometry
    
    return image



# close_crop('', img, geometry, options)

In [3]:
from slugify import Slugify
import glob
import os
from django.conf import settings
from fuzzywuzzy import fuzz
from django.core.files import File
import ipdb


def get_byline_image(self, force_new=False):
    slugify = Slugify(to_lower=True)
    if not force_new and self.byline_photo:
        return self.byline_photo
    imagefiles = glob.glob(settings.BYLINE_PHOTO_FOLDER + '/*.jpg')
    name = self.name.lower()
    name_last_first = re.sub(r'^(.*) (\S+)$', r'\2 \1', name)
    name_slug = slugify(name) + '.jpg'
    name_slug_reverse = slugify(name_last_first) + '.jpg'
    bestratio = 90
    bestmatch = None
    for path in imagefiles:
        filename = os.path.split(path)[1].lower()
        ratio = max(
            fuzz.ratio(filename, name_slug),
            fuzz.ratio(filename, name_slug_reverse)
        )
        if ratio > bestratio:
            bestmatch = path
            bestratio = ratio
            if ratio == 100:
                break
    if bestmatch:
        content = File(bestmatch)
        msg = 'found match: name:{}, img:{}, ratio:{} '.format(
            name_slug, bestmatch, ratio)
        #logger.debug(msg)
        ipdb.set_trace()
        img = ImageFile()
        img.upload_to = 'byline-photo'
        img.source_file.save(name_slug, content)
        img.autocrop()
        self.byline_photo = img
        self.save()
        return img
    
# Contributor.get_byline_image = get_byline_image
louise = Contributor.objects.filter(display_name__startswith='Louise').first()
img = louise.get_byline_image()


In [4]:
imgd = {'size': (img.full_width, img.full_height), 'source':img.source_file.path , 'options': {}} 

close_crop('', imgd , geometry, options)

696.57 615.0


{'options': {},
 'size': (200, 200),
 'source': '/srv/local.universitas.no/media/byline-photo/Louise-Faldalen-Prytz_TDp0vdH.jpg'}

In [5]:
img.source_file.path

'/srv/local.universitas.no/media/byline-photo/Louise-Faldalen-Prytz_TDp0vdH.jpg'